## Model loading and quantization

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
torch.manual_seed(42)

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
device

m:\python_projects\t-lite_CoT\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'cuda'

In [2]:
import gc

gc.collect()
torch.cuda.empty_cache()
gc.collect()

0

In [3]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM


model_name = "t-tech/T-lite-it-1.0"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_storage=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
)
model.config.use_cache = False

Loading checkpoint shards: 100%|██████████| 4/4 [00:36<00:00,  9.18s/it]


In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [5]:
prompt = "Сколько вертолетов может съесть человек?"
messages = [
    {"role": "system", "content": "Ты T-lite, виртуальный ассистент в Т-Технологии. Твоя задача - быть полезным диалоговым ассистентом."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=256
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

Вопрос о том, сколько вертолетов может съесть человек, является скорее шутливым или абсурдным. В реальности человек не может съесть вертолет из-за его размеров и материалов, из которых он изготовлен. Вертолеты состоят из металла, пластика и других материалов, которые не являются съедобными для человека. Если у вас есть другие вопросы или темы для обсуждения, дайте знать!


In [6]:
import gc

gc.collect()
torch.cuda.empty_cache()
gc.collect()

0

In [7]:
total_size = sum(param.numel() for param in model.parameters())
print(f"Number of parameters: {total_size / 1000000000:.2f}b")

Number of parameters: 2.72b


## Push quantized model on HF-hub

In [ ]:
from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv()

login(os.getenv("HF_TOKEN"))

In [ ]:
repo_name = "MilyaShams/T-lite-it-1.0_Q4_0"
model.push_to_hub(repo_name)

In [ ]:
tokenizer.push_to_hub(repo_name)